# Module 02: NPM Installation on Windows

**Difficulty**: ⭐  
**Estimated Time**: 45 minutes  
**Prerequisites**: 
- Module 01: Installation Methods Comparison
- Decision to use npm installation method
- Administrator access on Windows

## Learning Objectives

By the end of this notebook, you will be able to:

1. Install Node.js on Windows and verify the installation
2. Install n8n globally using npm and troubleshoot common errors
3. Configure environment variables for n8n on Windows
4. Create batch scripts for convenient n8n startup
5. Diagnose and fix common Windows-specific installation issues

## Why NPM Installation Matters

The npm installation method is the **fastest path from zero to running n8n** on Windows. Within 10 minutes, you can have:

- n8n installed and running
- Minimal resource overhead (100-320MB RAM)
- Direct access to configuration files
- No virtualization complexity

However, you'll encounter Windows-specific challenges that other platforms don't face:
- Permission errors in system directories
- Path configuration issues
- Native module build requirements

This module teaches you to navigate these issues confidently.

## Prerequisites Check

Before installing n8n, let's verify your system is ready.

In [ ]:
import subprocess
import sys
import os
from pathlib import Path

def check_administrator_rights():
    """
    Check if Python is running with administrator privileges.
    Required for global npm package installation.
    """
    try:
        # Check if we can write to a system directory
        return os.access('C:\\Windows\\System32', os.W_OK)
    except Exception:
        return False

def check_disk_space():
    """
    Check available disk space on C: drive.
    n8n requires about 1GB for installation.
    """
    try:
        result = subprocess.run(
            ['wmic', 'logicaldisk', 'where', 'DeviceID="C:"', 'get', 'FreeSpace'],
            capture_output=True,
            text=True,
            timeout=10
        )
        
        lines = result.stdout.strip().split('\n')
        if len(lines) > 1:
            free_bytes = int(lines[1].strip())
            free_gb = free_bytes / (1024**3)
            return free_gb
        return 0
    except Exception as e:
        return 0

print("=" * 60)
print("SYSTEM READINESS CHECK")
print("=" * 60)

# Check administrator rights
is_admin = check_administrator_rights()
admin_status = "✅ YES" if is_admin else "❌ NO"
print(f"Administrator Rights: {admin_status}")
if not is_admin:
    print("  ⚠️  Warning: You may need to run terminal as Administrator")

# Check disk space
free_space = check_disk_space()
space_status = "✅ Sufficient" if free_space > 2 else "⚠️  Low"
print(f"Free Disk Space: {free_space:.1f} GB - {space_status}")
if free_space < 2:
    print("  ⚠️  Warning: Consider freeing up disk space")

# Check Python version
python_version = sys.version.split()[0]
print(f"Python Version: {python_version}")

print("=" * 60)

## Step 1: Installing Node.js

Node.js is the JavaScript runtime that n8n requires. We need to install it before installing n8n.

### Downloading Node.js

1. Visit **https://nodejs.org/**
2. Download the **LTS (Long Term Support)** version
   - As of 2025, this includes versions 20.x and 22.x
   - Avoid "Current" version for stability
3. Look for the Windows Installer (.msi) - 64-bit version

### Installation Steps

Run the downloaded installer and follow these guidelines:

```
1. Welcome Screen → Click "Next"
2. License Agreement → Accept, click "Next"
3. Destination Folder → Use default (C:\Program Files\nodejs\)
4. Custom Setup → Keep all defaults INCLUDING:
   ✅ Node.js runtime
   ✅ npm package manager
   ✅ Add to PATH (CRITICAL)
   ✅ Online documentation shortcuts
5. Tools for Native Modules → Optional (we'll handle this later if needed)
6. Ready to Install → Click "Install"
7. Wait for installation (2-5 minutes)
8. Click "Finish"
```

### Why "Add to PATH" Matters

PATH is an environment variable that tells Windows where to find executable programs. When Node.js is in PATH:
- You can type `node` and `npm` in any terminal
- Without it, you'd need to type full paths like `C:\Program Files\nodejs\node.exe`

The installer handles this automatically if you keep defaults.

## Step 2: Verifying Node.js Installation

After installing Node.js, **close any open terminals** (this is critical!) and verify the installation.

In [ ]:
def check_nodejs_installation():
    """
    Verify Node.js and npm are installed and accessible.
    This simulates what you would check in Command Prompt.
    """
    print("=" * 60)
    print("NODE.JS INSTALLATION VERIFICATION")
    print("=" * 60)
    
    # Check Node.js
    try:
        result = subprocess.run(
            ['node', '--version'],
            capture_output=True,
            text=True,
            timeout=5
        )
        
        if result.returncode == 0:
            node_version = result.stdout.strip()
            print(f"✅ Node.js installed: {node_version}")
            
            # Check if version is compatible (v20.x or v22.x recommended)
            major_version = int(node_version.split('.')[0].replace('v', ''))
            if major_version >= 20:
                print(f"   ✅ Version {major_version}.x is compatible with n8n")
            else:
                print(f"   ⚠️  Version {major_version}.x may not be compatible")
                print("   Recommendation: Install Node.js 20.x or 22.x LTS")
        else:
            print("❌ Node.js not found or error occurred")
            print("   Action: Reinstall Node.js and ensure 'Add to PATH' is checked")
            return False
    except FileNotFoundError:
        print("❌ Node.js not found in PATH")
        print("   Action: Restart terminal or reinstall Node.js")
        return False
    except Exception as e:
        print(f"❌ Error checking Node.js: {e}")
        return False
    
    # Check npm
    try:
        result = subprocess.run(
            ['npm', '--version'],
            capture_output=True,
            text=True,
            timeout=5
        )
        
        if result.returncode == 0:
            npm_version = result.stdout.strip()
            print(f"✅ npm installed: {npm_version}")
        else:
            print("❌ npm not found")
            return False
    except Exception as e:
        print(f"❌ Error checking npm: {e}")
        return False
    
    print("=" * 60)
    print("✅ All checks passed! Ready to install n8n.")
    print("=" * 60)
    return True

# Run verification
check_nodejs_installation()

print("\n💡 If checks failed:")
print("   1. Close this notebook completely")
print("   2. Close any open Command Prompt/PowerShell windows")
print("   3. Open a NEW Command Prompt/PowerShell")
print("   4. Run: node --version")
print("   5. Run: npm --version")

## Step 3: Installing n8n via npm

With Node.js verified, we can now install n8n globally.

### The Installation Command

Open **Command Prompt** or **PowerShell** as **Administrator** and run:

```batch
npm install n8n -g
```

### Understanding the Command

- `npm`: Node Package Manager (installed with Node.js)
- `install`: Command to download and install a package
- `n8n`: The package name we want to install
- `-g`: Global flag (installs system-wide, not just in current directory)

### What to Expect

The installation process will:

1. **Download n8n** and all dependencies (~200MB)
2. **Compile native modules** (may show build warnings - usually safe to ignore)
3. **Create global command** (`n8n`) that you can run from anywhere
4. **Take 5-15 minutes** depending on internet speed and PC performance

### Installation Output Example

```
C:\Users\YourName> npm install n8n -g

npm WARN deprecated [various deprecation warnings - safe to ignore]

> [native module compilation messages]
> node-gyp rebuild

added 500 packages in 3m

50 packages are looking for funding
  run `npm fund` for details
```

## Step 4: Troubleshooting Common Installation Errors

Windows npm installations often encounter specific errors. Here's how to fix them.

In [ ]:
# Common error patterns and solutions

troubleshooting_guide = {
    "Error": [
        "EACCES: permission denied",
        "node-gyp rebuild failed",
        "Cannot find module '../build/Release/magic'",
        "Port 5678 already in use",
        "EPERM: operation not permitted"
    ],
    "Cause": [
        "Insufficient permissions for global install",
        "Missing C++ build tools for native modules",
        "Native module compilation failed",
        "Another application using port 5678",
        "File system permission issue"
    ],
    "Solution": [
        "Run Command Prompt/PowerShell as Administrator",
        "Install windows-build-tools (see below)",
        "Install windows-build-tools then reinstall n8n",
        "Change n8n port with N8N_PORT environment variable",
        "Take ownership of system folders (see below)"
    ]
}

import pandas as pd

troubleshooting_df = pd.DataFrame(troubleshooting_guide)

print("=" * 90)
print("COMMON NPM INSTALLATION ERRORS AND SOLUTIONS")
print("=" * 90)
print()
print(troubleshooting_df.to_string(index=False))
print()
print("=" * 90)

### Fix 1: Installing Windows Build Tools

If you see `node-gyp` errors, you need C++ build tools.

**Run in Administrator PowerShell/Command Prompt:**

```batch
npm install --global --production windows-build-tools
```

This installs:
- Python (required by node-gyp)
- Visual Studio Build Tools (C++ compiler)

**Time required**: 5-10 minutes  
**Action after**: Restart terminal and retry `npm install n8n -g`

### Fix 2: Taking Ownership of System Folders

Permission errors in `C:\Windows\System32\LogFiles\WMI` require taking ownership.

**PowerShell command (run as Administrator):**

```powershell
# Take ownership of WMI log folder
takeown /F C:\Windows\System32\LogFiles\WMI /R /D Y

# Grant administrators full control
icacls C:\Windows\System32\LogFiles\WMI /grant administrators:F /T
```

**Why this matters**: n8n tries to write to log directories that Windows restricts by default.

### Fix 3: Resolving Port Conflicts

If port 5678 is in use, identify what's using it:

```batch
netstat -ano | findstr :5678
```

This shows the Process ID (PID) using the port. You can:
- Kill that process: `taskkill /PID <number> /F`
- OR change n8n's port (covered in environment variables section)

## Step 5: Verifying n8n Installation

In [ ]:
def verify_n8n_installation():
    """
    Check if n8n is installed and accessible.
    """
    print("=" * 60)
    print("N8N INSTALLATION VERIFICATION")
    print("=" * 60)
    
    try:
        # Check n8n version
        result = subprocess.run(
            ['n8n', '--version'],
            capture_output=True,
            text=True,
            timeout=10
        )
        
        if result.returncode == 0:
            version = result.stdout.strip()
            print(f"✅ n8n installed successfully!")
            print(f"   Version: {version}")
            print()
            print("📍 n8n is installed at:")
            
            # Try to find n8n location
            where_result = subprocess.run(
                ['where', 'n8n'],
                capture_output=True,
                text=True,
                timeout=5
            )
            if where_result.returncode == 0:
                print(f"   {where_result.stdout.strip()}")
            
            print()
            print("✅ Ready to start n8n!")
            print("   Command: n8n")
            print("   Access at: http://localhost:5678")
            return True
        else:
            print("❌ n8n command not found")
            print("   Possible causes:")
            print("   - Installation failed (check error messages above)")
            print("   - Need to restart terminal")
            print("   - Need to run as Administrator")
            return False
            
    except FileNotFoundError:
        print("❌ n8n not found in PATH")
        print("   Actions:")
        print("   1. Verify installation completed without errors")
        print("   2. Restart terminal")
        print("   3. Check global npm packages: npm list -g --depth=0")
        return False
    except Exception as e:
        print(f"❌ Error: {e}")
        return False
    
    print("=" * 60)

# Run verification
verify_n8n_installation()

print("\n💡 To start n8n:")
print("   1. Open Command Prompt or PowerShell")
print("   2. Type: n8n")
print("   3. Wait for 'Editor is now accessible' message")
print("   4. Open browser to http://localhost:5678")

## Step 6: Understanding Environment Variables

Environment variables configure n8n without editing code. They control:
- Port number
- Security settings
- Database configuration
- Timezone
- And much more

### Two Ways to Set Environment Variables on Windows

#### Method 1: Temporary (Current Session Only)

**PowerShell:**
```powershell
$env:N8N_PORT="8080"
$env:N8N_BASIC_AUTH_ACTIVE="true"
n8n
```

**Command Prompt:**
```batch
set N8N_PORT=8080
set N8N_BASIC_AUTH_ACTIVE=true
n8n
```

**When to use**: Testing configurations, one-time changes

#### Method 2: Permanent (System-wide)

1. Press `Win + R`
2. Type: `sysdm.cpl` and press Enter
3. Click "Advanced" tab
4. Click "Environment Variables" button
5. Under "User variables" or "System variables", click "New"
6. Enter variable name and value
7. Click OK
8. **Restart all terminals** for changes to take effect

**When to use**: Permanent configuration you want every time

## Step 7: Creating Batch Scripts for Easy Startup

Batch scripts let you start n8n with all configuration in one click.

In [ ]:
# Generate a batch script for n8n startup

def create_startup_script():
    """
    Create a .bat file to start n8n with custom configuration.
    This helps avoid setting environment variables every time.
    """
    
    # Template for batch script
    batch_script = """@echo off
REM n8n Startup Script
REM Created by n8n Windows Automation Course

echo ========================================
echo Starting n8n...
echo ========================================
echo.

REM Set environment variables
SET N8N_PORT=5678
SET N8N_BASIC_AUTH_ACTIVE=true
SET N8N_BASIC_AUTH_USER=admin
SET N8N_BASIC_AUTH_PASSWORD=change_this_password
SET GENERIC_TIMEZONE=America/New_York
SET N8N_LOG_LEVEL=info

REM Optional: Set custom data directory
REM SET N8N_USER_FOLDER=C:\\n8n-data

echo Configuration:
echo   Port: %N8N_PORT%
echo   Auth: %N8N_BASIC_AUTH_ACTIVE%
echo   Timezone: %GENERIC_TIMEZONE%
echo.
echo Starting n8n...
echo Access at: http://localhost:%N8N_PORT%
echo.
echo Press Ctrl+C to stop n8n
echo ========================================
echo.

REM Start n8n
"%PROGRAMFILES%\\nodejs\\n8n.cmd"

REM If n8n exits, pause so you can see any error messages
echo.
echo n8n has stopped.
pause
"""
    
    print("=" * 70)
    print("BATCH SCRIPT TEMPLATE")
    print("=" * 70)
    print()
    print("Copy the script below and save as: start-n8n.bat")
    print("Recommended location: C:\\n8n\\start-n8n.bat")
    print()
    print("=" * 70)
    print(batch_script)
    print("=" * 70)
    print()
    print("📝 Customization Tips:")
    print("   1. Change N8N_BASIC_AUTH_PASSWORD to a strong password")
    print("   2. Update GENERIC_TIMEZONE to your timezone")
    print("   3. Change N8N_PORT if 5678 is already in use")
    print()
    print("▶️  To use:")
    print("   1. Create folder: C:\\n8n")
    print("   2. Save script as: C:\\n8n\\start-n8n.bat")
    print("   3. Double-click to run")
    print()
    print("💡 Create desktop shortcut:")
    print("   Right-click start-n8n.bat → Send to → Desktop (create shortcut)")
    
    return batch_script

# Generate and display the script
script_content = create_startup_script()

## Important Environment Variables for n8n

Here are the most commonly used environment variables:

In [ ]:
# Common n8n environment variables reference

env_vars_reference = {
    'Variable Name': [
        'N8N_PORT',
        'N8N_HOST',
        'N8N_BASIC_AUTH_ACTIVE',
        'N8N_BASIC_AUTH_USER',
        'N8N_BASIC_AUTH_PASSWORD',
        'GENERIC_TIMEZONE',
        'N8N_ENCRYPTION_KEY',
        'WEBHOOK_URL',
        'N8N_LOG_LEVEL'
    ],
    'Purpose': [
        'Port number for n8n interface',
        'Network interface to bind to',
        'Enable/disable authentication',
        'Username for basic auth',
        'Password for basic auth',
        'Your timezone for schedules',
        'Encryption key for credentials',
        'Base URL for webhooks',
        'Logging verbosity'
    ],
    'Default': [
        '5678',
        '0.0.0.0 (all)',
        'false',
        '(none)',
        '(none)',
        'America/New_York',
        '(auto-generated)',
        'http://localhost:5678',
        'info'
    ],
    'Example Value': [
        '8080',
        '127.0.0.1',
        'true',
        'admin',
        'MySecurePassword123!',
        'Asia/Singapore',
        'random-32-char-string',
        'https://mydomain.com',
        'debug'
    ]
}

env_df = pd.DataFrame(env_vars_reference)

print("=" * 100)
print("COMMON N8N ENVIRONMENT VARIABLES")
print("=" * 100)
print()
print(env_df.to_string(index=False))
print()
print("=" * 100)
print()
print("🔗 Full list: https://docs.n8n.io/reference/environment-variables/")

## Exercises

### Exercise 1: Verify Your Installation (Easy)

Complete these verification steps:

1. Open Command Prompt or PowerShell
2. Run `node --version` - record the version
3. Run `npm --version` - record the version
4. Run `n8n --version` - record the version
5. Check where n8n data will be stored: `echo %USERPROFILE%\.n8n`

In [ ]:
# Exercise 1: Record your installation details

installation_info = {
    'Node.js Version': '',  # Fill this in after running node --version
    'npm Version': '',      # Fill this in after running npm --version
    'n8n Version': '',      # Fill this in after running n8n --version
    'Data Directory': '',   # Fill this in with .n8n folder location
}

# After filling in the values, uncomment and run:
# print("My n8n Installation:")
# for key, value in installation_info.items():
#     print(f"{key}: {value}")

### Exercise 2: Create Custom Startup Script (Medium)

Modify the batch script template to:

1. Use port 8080 instead of 5678
2. Set your actual timezone (find yours: https://en.wikipedia.org/wiki/List_of_tz_database_time_zones)
3. Use your chosen username and strong password
4. Add a custom data directory: `C:\n8n-data`

Save it and test by double-clicking.

In [ ]:
# Exercise 2: Customize and validate your batch script

def validate_batch_script_config():
    """
    Helper to validate your batch script configuration.
    Fill in your values below.
    """
    
    my_config = {
        'port': 0,  # Your chosen port
        'timezone': '',  # Your timezone
        'username': '',  # Your username
        'data_folder': ''  # Your data directory
    }
    
    # Validation
    print("Configuration Validation:")
    print("=" * 50)
    
    # Check port
    if 1024 <= my_config['port'] <= 65535:
        print(f"✅ Port {my_config['port']} is valid")
    else:
        print(f"❌ Port must be between 1024-65535")
    
    # Check timezone format
    if '/' in my_config['timezone']:
        print(f"✅ Timezone '{my_config['timezone']}' format looks correct")
    else:
        print(f"⚠️  Timezone should be like 'America/New_York' or 'Asia/Singapore'")
    
    # Check username
    if len(my_config['username']) > 0:
        print(f"✅ Username set: {my_config['username']}")
    else:
        print(f"❌ Username cannot be empty")
    
    # Check data folder
    if my_config['data_folder'].startswith('C:'):
        print(f"✅ Data folder: {my_config['data_folder']}")
    else:
        print(f"⚠️  Data folder should be an absolute Windows path (C:\\...)")
    
    print("=" * 50)

# Uncomment and fill in your values:
# validate_batch_script_config()

### Exercise 3: Troubleshooting Simulation (Hard)

For each error scenario, identify:
1. The likely cause
2. The command(s) to diagnose the issue
3. The solution

**Scenario A**: You run `n8n` and get "'n8n' is not recognized as an internal or external command"

**Scenario B**: n8n starts but hangs at "Initializing n8n process" with no further output

**Scenario C**: During `npm install n8n -g`, you see "node-gyp rebuild failed" errors

**Scenario D**: n8n starts successfully, but when you access http://localhost:5678, the browser shows "This site can't be reached"

In [ ]:
# Exercise 3: Troubleshooting scenarios

def troubleshooting_exercise():
    scenarios = {
        'A': {
            'symptom': "'n8n' is not recognized",
            'cause': '',  # Fill in the likely cause
            'diagnostic_command': '',  # Fill in command to check
            'solution': ''  # Fill in the solution
        },
        'B': {
            'symptom': "Hangs at 'Initializing n8n process'",
            'cause': '',
            'diagnostic_command': '',
            'solution': ''
        },
        'C': {
            'symptom': "node-gyp rebuild failed",
            'cause': '',
            'diagnostic_command': '',
            'solution': ''
        },
        'D': {
            'symptom': "Browser can't reach localhost:5678",
            'cause': '',
            'diagnostic_command': '',
            'solution': ''
        }
    }
    
    # Fill in the scenarios above, then uncomment below to check your work:
    # for scenario_id, details in scenarios.items():
    #     print(f"\nScenario {scenario_id}: {details['symptom']}")
    #     print(f"  Cause: {details['cause']}")
    #     print(f"  Diagnostic: {details['diagnostic_command']}")
    #     print(f"  Solution: {details['solution']}")
    
    pass

troubleshooting_exercise()

## Summary

In this module, you learned:

### Installation Process

1. **Node.js Installation**: Download LTS version from nodejs.org, ensure "Add to PATH" is checked
2. **n8n Installation**: Single command `npm install n8n -g` installs globally
3. **Verification**: Use `node --version`, `npm --version`, `n8n --version` to verify

### Configuration

4. **Environment Variables**: Configure n8n behavior (port, auth, timezone, etc.)
5. **Batch Scripts**: Create `.bat` files for convenient startup with all settings
6. **Data Location**: n8n stores data at `%USERPROFILE%\.n8n` by default

### Troubleshooting

7. **Permission Errors**: Run as Administrator, take ownership of system folders
8. **Build Tool Errors**: Install windows-build-tools for native module compilation
9. **Port Conflicts**: Use `netstat` to identify conflicts, change port with N8N_PORT

### Key Takeaways

- ✅ npm installation is the fastest path to running n8n on Windows
- ✅ Most errors are Windows-specific and have documented solutions
- ✅ Batch scripts make daily usage convenient
- ✅ Environment variables provide flexible configuration

### What's Next

Now that n8n is installed, you can:

- **Module 04**: Complete initial configuration and create your first workflow
- **Module 06**: Implement security best practices
- **Module 08**: Set up automatic startup on Windows boot

Or, if you prefer Docker:

- **Module 03**: Install using Docker Desktop instead

### Additional Resources

- [n8n Environment Variables Reference](https://docs.n8n.io/reference/environment-variables/)
- [Node.js Official Documentation](https://nodejs.org/docs/)
- [npm Troubleshooting Guide](https://docs.npmjs.com/common-errors)
- [Windows Build Tools GitHub](https://github.com/felixrieseberg/windows-build-tools)